In [ ]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import network, block

### Exercise 1




#### Make [this test](/edit/session8/block.py) pass: `block.py:BlockTest:test_get_transactions`

In [ ]:
# Exercise 1

reload(block)
run(block.BlockTest('test_get_transactions'))

### Exercise 2
Check that the block with your previous transaction in fact has it


In [ ]:
# Exercise 2

from block import Block
from ecc import PrivateKey
from helper import decode_base58, hash256, little_endian_to_int
from network import SimpleNode, GetDataMessage, BLOCK_DATA_TYPE
from script import p2pkh_script
from tx import Tx
block_hash = bytes.fromhex('<block hash from class>')  # CHANGE
block_height = -1  # CHANGE
passphrase = b'<your passphrase here>'  # CHANGE
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(network="signet")
print(addr)
# convert the address to a ScriptPubKey using decode_base58 and p2pkh_script

# connect to signet.programmingbitcoin.com

# complete the handshake

# create a GetDataMessage

# add the BLOCK_DATA_TYPE with the block hash

# send the GetDataMessage

# wait for the Block

# get transactions in this block that have your ScriptPubKey

# print the first one serialized and hexadecimal


### Exercise 3




#### Make [this test](/edit/session8/network.py) pass: `network.py:SimpleNodeTest:test_get_block`

In [ ]:
# Exercise 3

reload(network)
run(network.SimpleNodeTest('test_get_block'))

### Exercise 4
You have been sent some unknown number of sats to your address on signet.

Send all of it back (minus fees) to `mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv` using only the networking protocol.

This should be a 1 input, 1 output transaction.

Turn on logging in `SimpleNode` if you need to debug


In [ ]:
# Exercise 4

from block import Block
from ecc import PrivateKey
from helper import decode_base58, hash160, hash256, little_endian_to_int
from network import GetDataMessage, SimpleNode, BLOCK_DATA_TYPE
from script import p2pkh_script
from tx import Tx, TxIn, TxOut
start_block_hex = '<insert from class>'  # CHANGE
block_hash = bytes.fromhex(block_hex)
passphrase = b'<get from session 2>'  # CHANGE
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(network="signet")
print(addr)
h160 = decode_base58(addr)
my_script_pubkey = p2pkh_script(h160)
target_address = 'mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv'
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
fee = 200  # fee in satoshis
# connect to signet.programmingbitcoin.com in signet mode

# complete the handshake

# get the block object using the get_block method of node

# initialize the utxos array

# grab the txs from the block using get_transactions

# there should be one transaction
if len(txs) != 1:
    raise RuntimeError("incorrect number of transactions")
# initialize the tx_ins array

# loop through the outputs of the transaction, enumerated

    # check if the output has the script pubkey we're looking for



        # record the amount from this output

# calculate the output amount (prev amount - fee)

# create TxOut

# create transaction on signet

# sign the only input in the tx

# print the tx's id

# send this signed transaction via the node
